<a href="https://colab.research.google.com/github/Sevenc1over/AI-class/blob/main/3%EC%A3%BC%EC%B0%A8/%ED%8C%8C%EC%9D%B4%ED%86%A0%EC%B9%98_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

요구사항 분석:

*   건포도 2가지를 분류해야한다.
*   input 데이터 영역은 실수형이다.
*   결측치 확인은 해야한다.



In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


# -----------------------------
# 1) 데이터 준비
# -----------------------------

df = pd.read_excel("/content/drive/MyDrive/Raisin_Dataset.xlsx").dropna()
df  # print(df) 와 동일

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Raisin_Dataset.xlsx'

레이블의 불균형 확인

In [ ]:
df['Class'].value_counts()
#코드는 Kecimen 450, Besni 450으로 전체 900개에서 반반이 나왔다.

그래프로 표현한 Class 레이블들에 갯

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.countplot(data = df, x="Class")
plt.xlabel("Class")        # 그래프 x 축 이름
plt.ylabel("Count")        # 그래프 y 축 이름
plt.title("건포도")

다음으로 결측치를 확인합니다.

In [ ]:
df.isnull().sum()
#위 코드를 통해 이번 데이터에는 결측치가 없다는것을 확인했다.

위 코드에 경우 결측치가 없기 때문에 문제가 없지만
df = df.dropna()
df.isnull().sum()
위 코드를 통해서 결측치가 있을 경우 행단위로 지우는 등에 행동을 해야한다.

In [ ]:
df = df.dropna()
df.isnull().sum()

In [ ]:
#컬럼명 확인
df.columns

위 과정을 통해서 데이터 레이블간 불균형과 결측치를 확인 할 수 있습니다. 이 과정이 끝났기에 아래 부터는 분류모델을 시작합니다.

In [ ]:
X=df.drop(columns=['Class'])
y=df["Class"]

In [ ]:
X

In [ ]:
y

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42

)

In [ ]:
#각 데이터의 숫자와 컬럼수 확인
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
# -----------------------------
# 2) 모델 구성
# -----------------------------
dt = DecisionTreeClassifier(random_state=42)
rf = RandomForestClassifier(n_estimators=200, random_state=42)
lr = LogisticRegression(max_iter=500)

In [ ]:
# -----------------------------
# 3) 모델 학습
# -----------------------------

dt.fit(X_train, y_train)
rf.fit(X_train, y_train)
lr.fit(X_train, y_train)

In [ ]:
# -----------------------------
# 4) 예측결과 생성
# -----------------------------


dt_y_pred = dt.predict(X_test)
rf_y_pred = rf.predict(X_test)
lr_y_pred = lr.predict(X_test)

In [ ]:
# 정답값 출력

y_test


In [ ]:

#예측값 출력

dt_y_pred

In [ ]:
# -----------------------------
# 5) 예측결과와 정답을 비교한 정확도 평가
# -----------------------------

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

dt_acc = accuracy_score(y_test, dt_y_pred)
rf_acc = accuracy_score(y_test, rf_y_pred )
lr_acc = accuracy_score(y_test, lr_y_pred)


print("=== DT Test Accuracy ===")
print(f"Decision Tree : {dt_acc:.4f}")
print (confusion_matrix(y_test,dt_y_pred))

print("=== RF Test Accuracy ===")
print(f"Random Forest : {rf_acc:.4f}")
print (confusion_matrix(y_test,rf_y_pred))

print("=== LR Test Accuracy ===")
print(f"Logistic Reg. : {lr_acc:.4f}")
print (confusion_matrix(y_test,lr_y_pred))